0. после выполнения крипта на загрузку данных в гит отпралялась база
0. очистка ticket base после полной загрузки
0. умный перебор: смотреть если перелет из точки А в точку B и в точку C (если нет из B в C то А в B не нужно)


0. когда точек отправления 2 и более сделеать
0. умная фильтрация по столбцам (все одинаковые фичи шли подряд)
0. составление маршрутов когда 2 и более точек
0. при фильтрации добавить duration (когда сравниваю две даты)
0. реализовать систему весов для сортировки
0. киллер фича, это учет часовых поясов аэропортов, даты начала рейса, продолжительности для высчитывания во сколько ты прилетаешь
0. продумать как json дозаписывать


In [1]:
#!pip install selenium

In [2]:
# https://support.travelpayouts.com/hc/ru/categories/200358578-API-%D0%B8-%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D0%B5
# https://support.travelpayouts.com/hc/ru/articles/203956163-Aviasales-API-%D0%B4%D0%BE%D1%81%D1%82%D1%83%D0%BF%D0%B0-%D0%BA-%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D0%BC-%D0%B4%D0%BB%D1%8F-%D1%83%D1%87%D0%B0%D1%81%D1%82%D0%BD%D0%B8%D0%BA%D0%BE%D0%B2-%D0%BF%D0%B0%D1%80%D1%82%D0%BD%D0%B5%D1%80%D1%81%D0%BA%D0%BE%D0%B9-%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D1%8B#15

In [1]:
ACCESS_TOKEN = 'b8848ff67e92768d2d1977862372d796'

origin = ['KWE']#'KWE'
final_destination = ['LED', 'DME', 'SVO', 'VKO', 'ZIA'] #'LED' 'KLO'
range_date = ['2020-01-07', '2020-01-15']
number_place = 2

time_sleep = 3
date_start, date_end = range_date[0], range_date[1]

url = 'http://api.travelpayouts.com/v2/prices/month-matrix'


In [2]:
import requests
import json
import codecs

from itertools import product
from tqdm import tqdm_notebook as tqdm

from datetime import datetime, timedelta
from time import sleep
import pandas as pd
from selenium import webdriver

add_zero = lambda x: '0' + str(x) if len(str(x)) < 2 else str(x)
# Используйте тут ваш путь к chromedriver!
chromedriver_path = 'chromedriver.exe'
driver = webdriver.Chrome(executable_path=chromedriver_path) # Этой командой открывается окно Chrome

url_routes = 'http://api.travelpayouts.com/data/routes.json'
routes = json.loads(requests.get(url_routes).text)
route_chain = list(set(map(lambda x: (x['departure_airport_iata'], x['arrival_airport_iata']), routes)))

In [293]:
def get_next_chain(arrivals, route_chain, final_destination, final = False):
    if final:
        next_chain = filter(lambda x: (x[0] in arrivals) & (x[1] in final_destination), route_chain)
    else:
        next_chain = filter(lambda x: x[0] in arrivals, route_chain)
    return next_chain

def get_arrivals(required_routes):
    if type(required_routes[0][0]) == str:
        arrivals = list(map(lambda x: x[1], required_routes))
    else: 
        arrivals = list(set(map(lambda x: x[-1][1], required_routes)))
    return arrivals

def get_product(required_routes, next_chain):
    if type(required_routes[0][0]) == str:
        required_routes = list(filter(lambda x: x[-2][1] == x[-1][0], product(required_routes, next_chain)))
    else: 
        required_routes = list(filter(lambda x: x[-2][-1][1] == x[-1][0], product(required_routes, next_chain)))
    return required_routes

def get_routes(origin, final_destination, route_chain, number_flights = 4):
    if number_flights == 1:
        required_routes = list(set(filter(lambda x: (x[0] in origin) & (x[1] in final_destination), route_chain)))
    else:
        required_routes = list(set(filter(lambda x: x[0] in origin, route_chain)))
        for n in range(1, number_flights):
            arrivals = get_arrivals(required_routes)
            if n + 1 == number_flights:
                next_chain = get_next_chain(arrivals, route_chain, final_destination, True)
            else:    
                next_chain = get_next_chain(arrivals, route_chain, final_destination)
            required_routes = get_product(required_routes, next_chain)           
    return required_routes

def flat_route(route):
    root = route
    result = []
    while type(root[-2]) != str:
        result.append(root[-1])
        root = root[-2]
        
    result.append(root)
    result.reverse()
    return result

In [295]:
required_routes = get_routes(origin, final_destination, route_chain, number_flights = 4)
required_routes = list(map(flat_route, required_routes))

[[('KWE', 'CAN'), ('CAN', 'CGK'), ('CGK', 'SIN'), ('SIN', 'DME')],
 [('KWE', 'CAN'), ('CAN', 'CGK'), ('CGK', 'HKG'), ('HKG', 'DME')],
 [('KWE', 'CAN'), ('CAN', 'CGK'), ('CGK', 'HKG'), ('HKG', 'SVO')],
 [('KWE', 'CAN'), ('CAN', 'CGK'), ('CGK', 'SGN'), ('SGN', 'DME')],
 [('KWE', 'CAN'), ('CAN', 'CGK'), ('CGK', 'SGN'), ('SGN', 'SVO')],
 [('KWE', 'CAN'), ('CAN', 'CGK'), ('CGK', 'NRT'), ('NRT', 'DME')],
 [('KWE', 'CAN'), ('CAN', 'CGK'), ('CGK', 'NRT'), ('NRT', 'SVO')],
 [('KWE', 'CAN'), ('CAN', 'CGK'), ('CGK', 'BKK'), ('BKK', 'SVO')],
 [('KWE', 'CAN'), ('CAN', 'CGK'), ('CGK', 'BKK'), ('BKK', 'DME')],
 [('KWE', 'CAN'), ('CAN', 'CGK'), ('CGK', 'HKT'), ('HKT', 'DME')],
 [('KWE', 'CAN'), ('CAN', 'CGK'), ('CGK', 'PEK'), ('PEK', 'LED')],
 [('KWE', 'CAN'), ('CAN', 'CGK'), ('CGK', 'PEK'), ('PEK', 'SVO')],
 [('KWE', 'CAN'), ('CAN', 'CGK'), ('CGK', 'PEK'), ('PEK', 'DME')],
 [('KWE', 'CAN'), ('CAN', 'CGK'), ('CGK', 'DOH'), ('DOH', 'DME')],
 [('KWE', 'CAN'), ('CAN', 'CGK'), ('CGK', 'AUH'), ('AUH', 'DME

In [5]:
def list_dates(datetime_start, datetime_end):
    delta = timedelta(days = 1)
    range_dates = []
    date = datetime_start
    while date <= datetime_end:
        range_dates.append(date)
        date = date + delta   
    return range_dates

def open_json(path):
    try:
        with open(path, encoding='utf-8', mode = 'r') as file:
            json_base = json.load(file)
    except FileNotFoundError:
        json_base = []
    return json_base

def write_json(path, json_base):
    with open(path, encoding='utf-8', mode = 'w') as file:
        json.dump(json_base, file)    

In [6]:
def get_destinations(routes, origin):
    destination_list = list(filter(lambda x: x['departure_airport_iata'] == origin, routes))
    destination_list = list(set(map(lambda x: x['arrival_airport_iata'],destination_list)))
    return destination_list

In [7]:
def update_browser_query(path, days = 2):
    browser_query = open_json(path)
    current_time = datetime.utcnow() - timedelta(days = days)
    browser_query = list(filter(lambda x: datetime.strptime(x['timestamp'], 
                                                            '%Y-%m-%d %H:%M:%S') > current_time, browser_query))  
    write_json(path, browser_query)  

def check_browser_query(date, origin, destination): 
    rewrite = True
    browser_query = open_json('browser_query.json')
    checker = list(filter(lambda x: ((x['date'] == date) & (x['origin'] == origin) & 
                                     (x['destination'] == destination)), browser_query))    
    if len(checker) != 0: rewrite = False          
    return rewrite   

def add_browser_query(date, origin, destination): 
    browser_query = open_json('browser_query.json')
    browser_query.append({'date':date, 'origin':origin, 
         'destination':destination, 'timestamp':datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')})
    write_json('browser_query.json', browser_query)


In [8]:
def update_price_base(path, days = 1):
    ticket_base = open_json(path)
    current_time = datetime.utcnow() - timedelta(days = days)   
    ticket_base = list(filter(lambda x: datetime.strptime(x['found_at'], 
                                                            '%Y-%m-%dT%H:%M:%S') > current_time, ticket_base))    
    write_json(path, ticket_base)       

def check_ticket_price(date, origin, destination): 
    rewrite = True
    ticket_base = open_json('ticket_base.json')       
    checker = list(filter(lambda x: ((x['depart_date'] == date) & (x['origin'] == origin) & 
                                     (x['destination'] == destination)), ticket_base))    
    if len(checker) != 0: rewrite = False       
    return rewrite

def get_ticket_price(date, origin, destination):
    ticket_base = open_json('ticket_base.json')
    ticket = list(filter(lambda x: ((x['depart_date'] == date) & (x['origin'] == origin) & 
                                     (x['destination'] == destination)), ticket_base))
    return ticket

def add_ticket_price(date, origin, destination, ticket): 
    ticket_base = open_json('ticket_base.json')
    ticket_base = ticket_base + ticket
    write_json('ticket_base.json', ticket_base)

In [9]:
def get_price_routes(datetime_start, datetime_end, origin, destination_list):
    
    update_price_base('ticket_base.json')
    update_browser_query('browser_query.json')
    
    parameters = list(product(list_dates(datetime_start, datetime_end), destination_list))

    for datetime_object, destination in tqdm(parameters, total=len(parameters), desc='send_query_for_' + origin):
        month, day = add_zero(datetime_object.month), add_zero(datetime_object.day)
        date = datetime_object.strftime('%Y-%m-%d')

        url_for_cash = f'https://www.aviasales.ru/search/{origin}{day}{month}{destination}1'
        
        if check_browser_query(date, origin, destination):
            driver.get(url_for_cash)
            sleep(time_sleep)
            while "Извините, доступ к поиску" in driver.page_source:
                print(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), 'sleeping 20 minutes')
                sleep(1200)
                driver.get(url_for_cash)
            
            add_browser_query(date, origin, destination)
                
    tickets = []
    for datetime_object, destination in tqdm(parameters, total=len(parameters), desc='get_price_from_' + origin):      
        date = datetime_object.strftime('%Y-%m-%d')
        if check_ticket_price(date, origin, destination):
            params = {'currency': 'rub', 'show_to_affiliates':'false', 'token' : ACCESS_TOKEN, 'one_way':'true',
                    'origin': origin, 'destination': destination, 'month':date}
            response = requests.get(url, params=params)
            if 'errors' not in json.loads(response.text)['data']:
                ticket = json.loads(response.text)['data']
                add_ticket_price(date, origin, destination, ticket)
                tickets = tickets + ticket  
        else:
            tickets = tickets + get_ticket_price(date, origin, destination)    
    return tickets

In [10]:
# f = codecs.open('airports.json', encoding='utf-8', mode='r')
# data = json.load(f)

In [11]:
tickets_list = []

datetime_start = datetime.strptime(date_start, '%Y-%m-%d')
datetime_end = datetime.strptime(date_end, '%Y-%m-%d')

destination_list = get_destinations(routes, origin)

tickets_list.append(get_price_routes(datetime_start, datetime_end, origin, destination_list))   

for change in range(number_place):
    ponts_list =  list(set(map(lambda x: x['destination'], tickets_list[change])))
    tickets = []
    for point in tqdm(ponts_list, desc=f'for_{change + 1}_point_destination'):
        if change + 1 < number_place: destination_list = get_destinations(routes, point) + final_destination
        else: destination_list = final_destination
        tickets = tickets + get_price_routes(datetime_start, datetime_end, point, destination_list)    
            
    tickets_list.append(tickets)  


KeyboardInterrupt: 

In [12]:
!git add ticket_base.json
!git add browser_query.json
!git commit -m "update jsons"
!git push origin master

[master c62d9d7] update jsons
 2 files changed, 2 insertions(+), 2 deletions(-)


To https://github.com/chylkov/aviasales_parser
   9c146ab..c62d9d7  master -> master


### Нужно порабатать с кодом

In [ ]:
df = pd.read_json('ticket_base.json')
df = df[(df['depart_date'] >= date_start) & (df['depart_date'] <= date_end)]

feature = list(df.columns).remove('found_at')

df.sort_values(by = 'found_at', ascending=False, inplace=True)
df.drop_duplicates(inplace = True, keep = 'first', subset = feature)

df.to_csv('routes_price.txt', index = False)

In [ ]:
route_feature = ['depart_date', 'origin', 'destination', 'distance', 
                 'duration', 'number_of_changes', 'value']

df_route = df[df['origin'] == origin][route_feature]

for change in range(number_place):
    
    if change == 0:
        df_route = df_route.merge(df[route_feature], 
                                           how = 'left', left_on = 'destination', 
                                           right_on = 'origin', suffixes = ['_from', ''])
        
        df_route.rename(columns={x:x + '_' + str(change + 1) for x in route_feature}, inplace = True)

        df_route = df_route[df_route['depart_date_from'] <= df_route['depart_date_' + str(change + 1)]]
    else:
        df_route = df_route.merge(df[route_feature], 
                                           how = 'left', left_on = 'destination_' + str(change), 
                                           right_on = 'origin')
        
        df_route.rename(columns={x:x + '_' + str(change + 1) for x in route_feature}, inplace = True)
        df_route = df_route[df_route['depart_date_' + str(change)] <= df_route['depart_date_' + str(change+1)]]


value_list = list(filter(lambda x: x[:5] == 'value', list(df_route.columns)))
df_route['total_value'] =  df_route[value_list].sum(axis = 1)

#df_route = df_route[(df_route['depart_date_from'] >= date_start) & (df_route['depart_date_to'] <= date_end)]

df_route.sort_values(by = ['total_value', 'depart_date_from'] , inplace = True)

df_route = df_route[df_route['destination' + str(change+1)].isin(final_destination)]

df_route.reset_index(drop=True, inplace = True)